In [ ]:
!pip install farm-haystack[colab] ##farm-haystack is a library that helps you build and deploy applications using large language models (LLMs). It provides tools to connect different components like models, databases, and data preprocessors into pipelines for tasks like question answering or information retrieval.
                                 #[colab]: This is an optional extra package installer for farm-haystack. It specifies that you want to install additional components specifically designed to work well in Google Colab environments.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.8/768.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from getpass import getpass
HF_token=getpass("Hugging Face Token: ") #HF_token = getpass("Hugging Face Token: ") uses the getpass function to display a message "Hugging Face Token: " and then reads the user's input without echoing the characters on the screen.

Hugging Face Token: ··········


In [ ]:
from haystack.nodes import PreProcessor,PromptModel,PromptTemplate,PromptNode ##PreProcessor: This class is responsible for cleaning and preparing text data before feeding it to other NLP models.
                                                                              #PromptModel: This class represents a large language model (LLM) that Haystack can interact with. It provides methods to send prompts (text instructions or questions) to the LLM and receive its response.
                                                                              #PromptTemplate: This class allows you to define templates for your prompts. Templates are essentially pre-defined text formats with placeholders for specific information.
                                                                              #PromptNode: This class is a building block for Haystack pipelines. It combines a PromptModel with a PromptTemplate and allows you to send queries or documents through the pipeline and receive the LLM's response based on the chosen template.

In [ ]:
from getpass import getpass
import json

def extract_text_from_json(json_file):
    try:
        with open(json_file, 'r') as file:
            text = ""
            for line in file:
                try:
                    data = json.loads(line)
                    for item in data:
                        if isinstance(data[item], str):
                            text += data[item] + " "
                        elif isinstance(data[item], dict):
                            for key in data[item]:
                                if isinstance(data[item][key], str):
                                    text += data[item][key] + " "
                        elif isinstance(data[item], list):
                            for element in data[item]:
                                if isinstance(element, str):
                                    text += element + " "
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
            return text.strip()
    except FileNotFoundError:
        print(f"File '{json_file}' not found.")
        return None

# Prompt user for JSON file path
json_file = input("Enter the path to the JSON file: ")

# Extract text from JSON file
extracted_text = extract_text_from_json(json_file)

if extracted_text:
    print("Extracted text:")
    print(extracted_text)


Enter the path to the JSON file: /content/kpi.jsonl
Extracted text:
OG_CALL_COUNT outgoing call count for a customers for past 30 days summerised OG_BUNDLE_CALL_COUNT The number of outgoing calls deducted from bundle subscription for a customer in the past month. OG_PAYG_CALL_COUNT The number of outgoing calls charged on a pay-as-you-go basis from the main balance for a customer in the past month. OG_FREE_CALL_COUNT The number of free outgoing calls for a customer in the past month. OG_IDD_PAYG_CALL_COUNT The number of international outgoing calls charged on a pay-as-you-go basis from the main balance for a customer in the past month. OG_IDD_GCC_CALL_COUNT The number of international outgoing calls to GCC countries charged on a pay-as-you-go basis from the main balance for a customer in the past month. OG_IDD_IBP_CALL_COUNT The number of international outgoing calls to IBP countries charged on a pay-as-you-go basis from the main balance for a customer in the past month. OG_IDD_VF_CALL_

In [ ]:
file_path = "/content/kpi.jsonl"
json_normalize_text= extract_text_from_json(file_path)
from haystack import Document
doc= Document(
    content=extracted_text,
    meta={"json_path":file_path} #meta: This is an optional dictionary where you can store additional metadata about the document.Metadata refers to data that provides information about other data. It describes various attributes of the data, such as its content, format, quality, source, or other characteristics. In essence, metadata gives context and meaning to the data it describes.
)

In [ ]:
docs = [doc]
processor = PreProcessor(

    clean_empty_lines=True,

    clean_whitespace=True,

    clean_header_footer=True,

    split_by="word",

    split_length=500,

    split_respect_sentence_boundary=True,

    split_overlap=0,

    language="en",
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
preprocessed_docs = processor.process(docs)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 69.84docs/s]


In [ ]:
from haystack.document_stores import InMemoryDocumentStore #Document Stores: Haystack uses document stores to manage and store the text documents you want to work with in your NLP pipelines.
                                                           #InMemoryDocumentStore: This specific class represents a document store that keeps all data in memory. It's a simple and fast option for experimentation or small datasets but not recommended for production due to memory limitations.

In [ ]:
document_store = InMemoryDocumentStore(use_bm25=True) #document_store = InMemoryDocumentStore(use_bm25=True) creates a new instance of the InMemoryDocumentStore class and assigns it to the variable document_store.

document_store.write_documents(preprocessed_docs)#document_store.write_documents(preprocessed_docs) writes a collection of preprocessed documents to the document_store.preprocessed_docs: This variable likely holds a list or collection of Haystack Document.

Updating BM25 representation...: 100%|██████████| 3/3 [00:00<00:00, 1632.02 docs/s]


In [ ]:
from haystack import Pipeline #A Haystack pipeline combines different NLP components (like retrieval, reading, and question answering) into a single processing unit.

from haystack.nodes import BM25Retriever #from haystack.nodes import BM25Retriever imports the BM25Retriever class from Haystack. This class represents a retriever component that uses the BM25 algorithm to find documents relevant to a user query.



In [ ]:
def prompt_retriever_config():
    top_k = int(input("Enter the number of documents to retrieve (default: 2): ") or "2") #This parameter sets the maximum number of documents the retriever will return for a given query.
    return top_k

# Prompt user for retriever configuration
top_k = prompt_retriever_config()

Enter the number of documents to retrieve (default: 2): 2


In [ ]:
retriever = BM25Retriever(document_store, top_k=top_k)

-The script prompts the user to input the path to a JSON file containing prompt templates.
-It loads the templates from the file and displays them to the user.
-The user selects a template by entering its index.
-The selected template is then used in the script.

In [ ]:
import json

# Define a function to load prompt templates from a JSON file
def load_prompt_templates(template_file):
    try:
        with open(template_file, 'r') as file:
            templates = json.load(file)
        return templates
    except FileNotFoundError:
        print(f"File '{template_file}' not found.")
        return None

# Prompt user for the path to the template file
template_file = input("Enter the path to the template JSON file: ")

# Load prompt templates
prompt_templates = load_prompt_templates(template_file)

if prompt_templates:
    # Display available prompt templates
    print("Available Prompt Templates:")
    for i, template in enumerate(prompt_templates, 1):
        print(f"{i}. {template}")

    # Prompt user to select a template
    template_index = int(input("Enter the index of the desired template: ")) - 1
    selected_template = prompt_templates[template_index]
    qa_template = PromptTemplate(prompt=selected_template)
else:
    print("No prompt templates loaded. Exiting...")
    exit()

In [ ]:
qa_template = PromptTemplate(prompt=

"""
Make sure the response provided is strictly in the format given below."

Given below is a format for the RAG model prompt and the desired format for the response:

Query Format:

"Get customers who had free outgoing calls in the past month and the total duration of those calls exceeded 60 minutes."

Desired Response Format:

“OG_FREE_CALL_COUNT > 0 AND OG_FREE_CALL_MOU > 60”

Context: {join(documents)};
Question: {query}
""")


In [ ]:
prompt_node = PromptNode(

    model_name_or_path="mistralai/Mixtral-8x7B-Instruct-v0.1",

    api_key=HF_token,

    default_prompt_template=qa_template,

    max_length=int(input("Enter the maximum length of input query (default: 500): ") or "500"), #max_length=500 (optional argument): This parameter sets the maximum length of the response generated by the LLM. Here, it's set to 500 tokens, limiting the LLM's output to a maximum of 500 words or units of text.

    model_kwargs={"model_max_length": int(input("Enter the maximum length for model response (default: 11000): ") or "11000")} #model_kwargs={"model_max_length": 5000} (optional argument): This argument passes additional keyword arguments to the underlying LLM. Here, it sets the model_max_length parameter within the model itself to 5000 tokens, which is a higher limit than the overall max_length of 500 imposed on the final response. This way, the LLM can internally process information up to 5000 tokens but will ultimately return a response trimmed to 500 tokens based on the first argument max_length.
)

Enter the maximum length of input query (default: 500): 400
Enter the maximum length for model response (default: 11000): 9000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
rag_pipeline = Pipeline()#Here, a new pipeline object named rag_pipeline is created using the Pipeline class. This pipeline is presumably intended to perform a sequence of tasks or processing steps.

rag_pipeline.add_node(component=retriever, name="retriever", inputs=["Query"])#This line adds a node to the pipeline named "retriever". This node likely represents a component responsible for retrieving information relevant to a given query. The retriever variable seems to refer to an object or function representing this component. The inputs parameter specifies that the input data for this node is expected to come from a source named "Query".

rag_pipeline.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])#Another node is added to the pipeline, named "prompt_node". This node seems to represent a component responsible for prompting users or generating prompts based on retrieved information. The prompt_node variable likely refers to an object or function representing this component. The inputs parameter specifies that the input data for this node is expected to come from the node named "retriever", indicating that the output of the retriever node serves as input to the prompt node.

In [ ]:
from pprint import pprint #The pprint module in Python stands for "pretty print." It provides a way to format Python data structures in a more human-readable and visually appealing way when printed to the console.
print_answer = lambda out: pprint(out["results"][0].strip()) #This code defines a lambda function print_answer that takes an input dictionary out. The lambda function assumes that out contains a key "results" which holds a list of strings. It then prints the first string from this list after stripping any leading or trailing whitespace.

In [ ]:
print_answer(rag_pipeline.run(query="Get customers who have only 5 incoming calls in the past 3 months and the total duration of those calls exceeded 30 minutes."))

'Answer:\n\n"IC_CALL_COUNT = 5 AND IC_CALL_MOU > 30"'
